# Monte Carlo Markov Chain (MCMC) Simulation with Metropolis-Hastings Algorithm

This tutorial demonstrates how to implement a Monte Carlo Markov Chain using the Metropolis-Hastings algorithm for parameter estimation in cosmology. We will use the Cosmic Microwave Background (CMB) power spectra data simulated using the `camb` library.

## Key Concepts:
- **CAMB**: A software package used to simulate the CMB power spectra.
- **Metropolis-Hastings Algorithm**: A method for obtaining a sequence of random samples from a probability distribution for which direct sampling is difficult.
- **Likelihood Computation**: Using the CMB power spectra to compute likelihoods.
- **Priors and Proposals**: Setting up the prior distributions and proposal distributions for our MCMC sampler.


In [12]:
# Install the camb package if not already installed
# !pip install camb

# Import necessary libraries
import camb
import numpy as np
import os
import pickle
import time
from camb.model import CAMBParamRangeError
from scipy.stats import norm
from scipy.io import FortranFile
import scipy.linalg

# from numba import jit


## Define the Power Spectrum Calculation Function 

In [9]:
def calculate_power_spectra(params, lensed=True, accuracy_boost=1, l_sample_boost=1, l_accuracy_boost=1):
    """
    Calculates the CMB power spectra given cosmological parameters.

    Parameters:
    - params (dict): Cosmological parameters
    - lensed (bool): Whether to use lensed spectra
    - accuracy_boost (float): Increase accuracy of calculations
    - l_sample_boost (float): Increase l sample boost
    - l_accuracy_boost (float): Increase l accuracy boost
    """
    # Additional settings for lower accuracy to speed up simulations
    params.update({
        'AccuracyBoost': accuracy_boost,
        'lSampleBoost': l_sample_boost,
        'lAccuracyBoost': l_accuracy_boost,
        'DoLateRadTruncation': True
    })

    # Set and get results from CAMB
    pars = camb.set_params(**params)
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')

    # Select either lensed or unlensed spectra
    cl = powers['total'] if lensed else powers['unlensed_scalar']

    # Remove monopole and dipole terms and return TT, TE, and EE spectra
    cl_cut = cl[2:]
    return cl_cut[:, 0], cl_cut[:, 3], cl_cut[:, 1]

## Define the Parameter Processing Function

In [10]:
def process_params_and_calculate_spectra(array):
    """
    Adjusts raw parameter array values and calculates power spectra.

    Parameters:
    - array (numpy.ndarray): Array of parameter values
    """
    array = array.copy()
    array[2] /= 100
    array[5] = np.exp(array[5]) / 1e10
    params = {
        'ombh2': array[0],
        'omch2': array[1],
        'cosmomc_theta': array[2],
        'tau': array[3],
        'ns': array[4],
        'As': array[5],
        'mnu': 0.06,
        'omk': 0,
        'halofit_version': 'mead',
        'lmax': 2800
    }
    return calculate_power_spectra(params)


## Define the Likelihood Calculation Function

In [23]:

class PlanckLitePy:
    def __init__(self, data_directory='./MCMC/data', year=2018, spectra='TT', use_low_ell_bins=False):
        '''
        data_directory = path from where you are running this to the folder
          containing the planck2015/8_low_ell and planck2015/8_plik_lite data
        year = 2015 or 2018
        spectra = TT for just temperature or TTTEEE for temperature (TT),
          E mode (EE) and cross (TE) spectra
        use_low_ell_bins = True to use 2 low ell bins for the TT 2<=ell<30 data
          or False to only use ell>=30
        '''
        self.year=year
        self.spectra=spectra
        self.use_low_ell_bins=use_low_ell_bins #False matches Plik_lite - just l>=30

        if self.use_low_ell_bins:
            self.nbintt_low_ell=2
            self.plmin_TT=2
        else:
            self.nbintt_low_ell=0
            self.plmin_TT=30
        self.plmin=30
        self.plmax=2508
        self.calPlanck=1

        if year==2015:
            self.data_dir=data_directory+'/planck2015_plik_lite/'
            version=18
        elif year==2018:
            self.data_dir=data_directory+'/planck2018_plik_lite/'
            version=22
        else:
            print('Year must be 2015 or 2018')
            return 1

        if spectra=='TT':
            self.use_tt=True
            self.use_ee=False
            self.use_te=False
        elif spectra=='TTTEEE':
            self.use_tt=True
            self.use_ee=True
            self.use_te=True
        else:
            print('Spectra must be TT or TTTEEE')
            return 1

        self.nbintt_hi = 215 #30-2508   #used when getting covariance matrix
        self.nbinte = 199 #30-1996
        self.nbinee = 199 #30-1996
        self.nbin_hi=self.nbintt_hi+self.nbinte+self.nbinee

        self.nbintt=self.nbintt_hi+self.nbintt_low_ell #mostly want this if using low ell
        self.nbin_tot=self.nbintt+self.nbinte+self.nbinee

        self.like_file = self.data_dir+'cl_cmb_plik_v'+str(version)+'.dat'
        self.cov_file  = self.data_dir+'c_matrix_plik_v'+str(version)+'.dat'
        self.blmin_file = self.data_dir+'blmin.dat'
        self.blmax_file = self.data_dir+'blmax.dat'
        self.binw_file = self.data_dir+'bweight.dat'

        # read in binned ell value, C(l) TT, TE and EE and errors
        # use_tt etc to select relevant parts
        self.bval, self.X_data, self.X_sig=np.genfromtxt(self.like_file, unpack=True)
        self.blmin=np.loadtxt(self.blmin_file).astype(int)
        self.blmax=np.loadtxt(self.blmax_file).astype(int)
        self.bin_w=np.loadtxt(self.binw_file)

        if self.use_low_ell_bins:
            self.data_dir_low_ell=data_directory+'/planck'+str(year)+'_low_ell/'
            self.bval_low_ell, self.X_data_low_ell, self.X_sig_low_ell=np.genfromtxt(self.data_dir_low_ell+'CTT_bin_low_ell_'+str(year)+'.dat', unpack=True)
            self.blmin_low_ell=np.loadtxt(self.data_dir_low_ell+'blmin_low_ell.dat').astype(int)
            self.blmax_low_ell=np.loadtxt(self.data_dir_low_ell+'blmax_low_ell.dat').astype(int)
            self.bin_w_low_ell=np.loadtxt(self.data_dir_low_ell+'bweight_low_ell.dat')

            self.bval=np.concatenate((self.bval_low_ell, self.bval))
            self.X_data=np.concatenate((self.X_data_low_ell, self.X_data))
            self.X_sig=np.concatenate((self.X_sig_low_ell, self.X_sig))

            self.blmin_TT=np.concatenate((self.blmin_low_ell, self.blmin+len(self.bin_w_low_ell)))
            self.blmax_TT=np.concatenate((self.blmax_low_ell, self.blmax+len(self.bin_w_low_ell)))
            self.bin_w_TT=np.concatenate((self.bin_w_low_ell, self.bin_w))

        else:
            self.blmin_TT=self.blmin
            self.blmax_TT=self.blmax
            self.bin_w_TT=self.bin_w


        self.fisher=self.get_inverse_covmat()

    def get_inverse_covmat(self):
        # Read full covmat
        f = FortranFile(self.cov_file, 'r')
        covmat = f.read_reals(dtype=float).reshape((self.nbin_hi, self.nbin_hi))
        covmat = np.maximum(covmat, covmat.T)  # Make the matrix symmetric

        # Define a dictionary to map conditions to calculations
        conditions = {
            (True, False, False): (self.nbintt_hi, 0),
            (False, False, True): (self.nbinte, self.nbintt_hi),
            (False, True, False): (self.nbinee, self.nbintt_hi + self.nbinte),
            (True, True, True): (self.nbin_hi, 0)
        }

        # Select relevant covmat
        bin_no, start = conditions.get((self.use_tt, self.use_ee, self.use_te), (None, None))
        if bin_no is None:
            print("not implemented")
            return

        end = start + bin_no
        cov = covmat[start:end, start:end]

        # Invert high ell covariance matrix (cholesky decomposition should be faster)
        fisher = scipy.linalg.cho_solve(scipy.linalg.cho_factor(cov), np.identity(bin_no)).T

        if self.use_low_ell_bins:
            bin_no += self.nbintt_low_ell
            inv_covmat_with_lo = np.zeros(shape=(bin_no, bin_no))
            inv_covmat_with_lo[0:2, 0:2] = np.diag(1. / self.X_sig_low_ell**2)
            inv_covmat_with_lo[2:, 2:] = fisher
            fisher = inv_covmat_with_lo

        return fisher



    # def loglike(self, Cltt, Clte, Clee, ellmin=2):
        

    #     # If the input is Dl's, convert to Cl's
    #     #convert model Dl's to Cls then bin them
    #     # ls=np.arange(len(Dltt))+ellmin
    #     # fac=ls*(ls+1)/(2*np.pi)
    #     # Cltt=Dltt/fac
    #     # Clte=Dlte/fac
    #     # Clee=Dlee/fac



    def binning(self, Cl, blmin, blmax, bin_w, nbins, ellmin):
        Cl_bin = np.zeros(nbins)
        for i in range(nbins):
            Cl_bin[i] = np.sum(Cl[blmin[i]+self.plmin-ellmin:blmax[i]+self.plmin+1-ellmin]*bin_w[blmin[i]:blmax[i]+1])
        return Cl_bin

    def select_diff_vec(self, Y):
        conditions = {
            (True, False, False): (self.nbintt, 0),
            (False, False, True): (self.nbinte, self.nbintt),
            (False, True, False): (self.nbinee, self.nbintt + self.nbinte),
            (True, True, True): (self.nbin_tot, 0)
        }
        bin_no, start = conditions.get((self.use_tt, self.use_ee, self.use_te), (None, None))
        if bin_no is None:
            print("not implemented")
            return None
        end = start + bin_no
        return Y[start:end]

    def loglike(self, Dltt, Dlte, Dlee, ellmin=2):
        # Convert model Dl's to Cls then bin them
        ls = np.arange(len(Dltt)) + ellmin
        fac = ls * (ls + 1) / (2 * np.pi)
        Cltt, Clte, Clee = Dltt / fac, Dlte / fac, Dlee / fac

        # Bin the Cls
        Cltt_bin = self.binning(Cltt, self.blmin_TT, self.blmax_TT, self.bin_w_TT, self.nbintt, ellmin)
        Clte_bin = self.binning(Clte, self.blmin, self.blmax, self.bin_w, self.nbinte, ellmin)
        Clee_bin = self.binning(Clee, self.blmin, self.blmax, self.bin_w, self.nbinee, ellmin)

        # Construct the model
        X_model = np.zeros(self.nbin_tot)
        X_model[:self.nbintt] = Cltt_bin / self.calPlanck**2
        X_model[self.nbintt:self.nbintt+self.nbinte] = Clte_bin / self.calPlanck**2
        X_model[self.nbintt+self.nbinte:] = Clee_bin / self.calPlanck**2

        # Calculate the difference vector
        Y = self.X_data - X_model
        diff_vec = self.select_diff_vec(Y)
        if diff_vec is None:
            return None

        return -0.5 * diff_vec.dot(self.fisher.dot(diff_vec))


## Define the Prior and Proposal Distributions

In [24]:
# Define the prior distributions. This is what one would expect to see after the burn-in steps. 
priors = [
    norm(loc=0.0224, scale=np.sqrt(1)*0.00015),
    norm(loc=0.12, scale=np.sqrt(1)*0.0015),
    norm(loc=1.04109, scale=np.sqrt(1)*0.0004),
    norm(loc=0.055, scale=np.sqrt(1)*0.009),
    norm(loc=0.965, scale=np.sqrt(1)*0.007),
    norm(loc=3.05, scale=np.sqrt(1)*0.01)
]

# Define the proposal distribution function. This is an example value that works well after the burn-in step. 
a = 10
def proposal_distribution(current_state):
    step_sizes = np.array([
        0.0001/a, 0.001/a, 0.0002/a, 0.003/15, 0.002/a, 0.01/a
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step

## Implement the MCMC Sampler

In [25]:
class MCMC_MH:
    def __init__(self, likelihood, proposal_distribution, priors, num_samples, num_chains, stepsize=0.5, burnin_ratio=0.05, resume=False):
        self.likelihood = likelihood
        self.proposal_distribution = proposal_distribution
        self.priors = priors
        self.num_samples = num_samples
        self.num_chains = num_chains
        self.stepsize = stepsize
        self.burnin = int(burnin_ratio * num_samples)
        self.checkpoint_interval = 250
        self.resume = resume
        self.R_minus_one = None

        self.count = 0

        self.curr_state = self.generate_initial_states()
        self.curr_likeli = [likelihood.loglike(*process_params_and_calculate_spectra(state)) for state in self.curr_state]
        self.samples = [[] for _ in range(num_chains)]
        self.likelihoods = [[] for _ in range(num_chains)]
        self.accepteds = [0 for _ in range(num_chains)]
        self.Rminusones = []


        if resume:
            self.load_checkpoint()
        elif os.path.exists('./checkpoints/checkpoint_burnin.pkl') or os.path.exists('./checkpoints/checkpoint_production.pkl'):
            raise ValueError("The checkpoint files 'checkpoint_burnin.pkl' and 'checkpoint_production.pkl' should not exist when starting a new run.")

    def generate_initial_states(self):
        initial_states = []
        for _ in range(self.num_chains):
            state = [prior.rvs() for prior in self.priors]
            initial_states.append(state)
        return initial_states

    def save_checkpoint(self, iteration, burn_in=False):
        if not os.path.exists('./checkpoints'):
            os.makedirs('./checkpoints')
        filename = f'./checkpoints/checkpoint_{"burnin" if burn_in else "production"}.pkl'
        with open(filename, 'wb') as f:
            pickle.dump((self.curr_state, self.samples, self.likelihoods, self.Rminusones, self.count), f)

    def load_checkpoint(self):
        checkpoint_file_prod = './checkpoints/checkpoint_production.pkl'
        checkpoint_file_burnin = './checkpoints/checkpoint_burnin.pkl'
        
        if os.path.exists(checkpoint_file_prod):
            with open(checkpoint_file_prod, 'rb') as f:
                self.curr_state, self.samples, self.likelihoods, self.Rminusones, self.count = pickle.load(f)
        elif os.path.exists(checkpoint_file_burnin):
            with open(checkpoint_file_burnin, 'rb') as f:
                self.curr_state, self.samples, self.likelihoods, self.Rminusones, self.count = pickle.load(f)
        else:
            raise ValueError("No checkpoint files found to resume from.")

    def burn_in(self):
        start_time = time.time()
        if self.resume:
            self.load_checkpoint()

        for i in range(self.burnin):
            for j in range(self.num_chains):
                self.mcmc_updater(j)
            if (i + 1) % self.checkpoint_interval == 0:
                self.save_checkpoint(i + 1, burn_in=True)
                elapsed_time = time.time() - start_time
                print(f"Completed {i+1} burn-in steps in {elapsed_time} seconds")
            self.count += 1

    def mcmc_updater(self, chain_index):
        proposal_state = self.proposal_distribution(self.curr_state[chain_index])
        proposal_state = [max(0, value) for value in proposal_state]

        try:
            prop_loglikeli = self.likelihood.loglike(*process_params_and_calculate_spectra(proposal_state))
        except CAMBParamRangeError:
            prop_loglikeli = -np.inf
        except Exception as e:
            print(f"An error occurred: {e}")
            prop_loglikeli = -np.inf

        accept_crit = prop_loglikeli - self.curr_likeli[chain_index]
        accept_threshold = np.log(np.random.uniform(0, 1))

        if accept_crit > accept_threshold:
            self.curr_state[chain_index], self.curr_likeli[chain_index] = proposal_state, prop_loglikeli


        return self.curr_state[chain_index], self.curr_likeli[chain_index]

    def metropolis_hastings(self):
        start_time = time.time()
        self.burn_in()

        if self.resume:
            self.load_checkpoint()

        print(self.count, self.num_samples)
        for i in range(self.count, self.num_samples):
            for j in range(self.num_chains):
                self.curr_state[j], self.curr_likeli[j] = self.mcmc_updater(j)
                self.samples[j].append(self.curr_state[j])
                self.likelihoods[j].append(self.curr_likeli[j])

            if (i + 1) % self.checkpoint_interval == 0:
                self.save_checkpoint(i + 1)
                print(f"Completed {i+1} steps in {time.time() - start_time} seconds")
            if (i + 1) % 50 == 0:
                self.calculate_convergence()
            
            self.count += 1

        return self.samples

    def calculate_convergence(self):
        chain_means = [np.mean(np.array(chain), axis=0) for chain in self.samples]
        chain_vars = [np.var(np.array(chain), axis=0, ddof=1) for chain in self.samples]

        grand_mean = np.mean(chain_means, axis=0)

        B = np.sum((chain_means - grand_mean)**2, axis=0) * len(self.samples[0]) / (len(self.samples) - 1)
        W = np.mean(chain_vars, axis=0)

        var_plus = (len(self.samples[0]) - 1) / len(self.samples[0]) * W + B / len(self.samples[0])
        self.R_minus_one = (var_plus / W - 1).mean()
        print(f'R-1 statistic: {self.R_minus_one}')      
        self.Rminusones.append(self.R_minus_one)

In [26]:
likelihood_function = PlanckLitePy()

mcmc = MCMC_MH(likelihood_function, proposal_distribution, priors, 10, num_chains=4, stepsize=0.1, burnin_ratio=0.00, resume=False)
mcmc.metropolis_hastings()

0 10


KeyboardInterrupt: 